In [ ]:
# zillow data scratchpad for regression-project

In [12]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.preprocessing import PolynomialFeatures

from acquire import get_zillow_data

# Acquire Zillow Data

In [13]:
# getting my zillow data using my function 'get_zillow_data' from my acquire file
df = get_zillow_data()

In [14]:
df.head() # check_yo_head

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate
0,14634203,2026522,1.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,NaN,NaN,6.059001e+13,625,-0.061973,2017-08-04
1,11721753,616260,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,NaN,NaN,6.037220e+13,1018,0.017785,2017-07-21
2,11289917,2061546,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
3,11637029,2554497,1.0,NaN,NaN,2.0,3.0,NaN,9.0,2.0,...,810694.0,2016.0,554156.0,9407.01,Y,13.0,6.037701e+13,1734,-0.093399,2017-07-28
4,11705026,1834372,NaN,NaN,NaN,1.0,2.0,NaN,6.0,1.0,...,35606.0,2016.0,23624.0,543.69,NaN,NaN,6.037232e+13,1772,-0.146056,2017-06-30


In [15]:
# taking a look at the data info
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38622 entries, 0 to 38621
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      38622 non-null  int64  
 1   id                            38622 non-null  int64  
 2   airconditioningtypeid         12224 non-null  float64
 3   architecturalstyletypeid      101 non-null    float64
 4   basementsqft                  27 non-null     float64
 5   bathroomcnt                   38622 non-null  float64
 6   bedroomcnt                    38622 non-null  float64
 7   buildingclasstypeid           0 non-null      float64
 8   buildingqualitytypeid         23706 non-null  float64
 9   calculatedbathnbr             38463 non-null  float64
 10  decktypeid                    333 non-null    float64
 11  finishedfloor1squarefeet      3339 non-null   float64
 12  calculatedfinishedsquarefeet  38509 non-null  float64
 13  f

In [16]:
# looking at all the columns in the data
df.columns

Index(['parcelid', 'id', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [17]:
# looking at the number of rows and columns in the zillow data
df.shape

(38622, 62)

In [18]:
# some quick stats on the zillow data
df.describe()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id.1,logerror
count,3.862200e+04,3.862200e+04,12224.000000,101.000000,27.000000,38622.000000,38622.000000,0.0,23706.000000,38463.000000,...,84.0,3.857000e+04,3.862100e+04,38622.0,3.862100e+04,38621.000000,1268.000000,3.847800e+04,38622.000000,38622.000000
mean,1.310495e+07,1.500394e+06,1.900196,7.564356,639.703704,2.273743,3.010512,NaN,6.610394,2.282739,...,1.0,1.907161e+05,4.965779e+05,2016.0,3.061136e+05,6042.953623,14.070978,6.050206e+13,52116.933794,0.015917
std,3.700741e+06,8.591808e+05,3.112632,3.232386,445.977907,0.950588,1.029506,NaN,1.694360,0.941702,...,0.0,2.384221e+05,6.326909e+05,0.0,4.583442e+05,7366.777380,1.482764,2.126099e+11,12046.770714,0.172150
min,1.071186e+07,3.490000e+02,1.000000,2.000000,100.000000,0.000000,0.000000,NaN,1.000000,1.000000,...,1.0,1.290000e+02,1.000000e+03,2016.0,1.000000e+03,19.920000,7.000000,6.037101e+13,625.000000,-4.453964
25%,1.154805e+07,7.599932e+05,1.000000,7.000000,349.000000,2.000000,2.000000,NaN,6.000000,2.000000,...,1.0,8.451250e+04,2.090510e+05,2016.0,8.702300e+04,2720.440000,14.000000,6.037400e+13,41836.250000,-0.023433
50%,1.260418e+07,1.507182e+06,1.000000,7.000000,604.000000,2.000000,3.000000,NaN,7.000000,2.000000,...,1.0,1.369250e+05,3.635160e+05,2016.0,2.080000e+05,4464.540000,15.000000,6.037651e+13,52156.500000,0.006802
75%,1.429905e+07,2.242966e+06,1.000000,7.000000,743.000000,3.000000,4.000000,NaN,8.000000,3.000000,...,1.0,2.199940e+05,5.778040e+05,2016.0,3.761050e+05,6984.240000,15.000000,6.059052e+13,62495.750000,0.037605
max,1.676878e+08,2.982274e+06,13.000000,21.000000,1969.000000,11.000000,12.000000,NaN,12.000000,11.000000,...,1.0,1.142179e+07,2.385837e+07,2016.0,1.635060e+07,276797.830000,15.000000,6.111009e+13,72844.000000,3.175688


### Acquire Takeaways
- way too many columns (62) in the zillow data that are not needed
- will need to either drop columns or jsut select the needed columns to use for the project
- will need to also rename columns in order to make them eaiser to read and use
- will need to drop remaining null values in the zillow data

# Prep

In [19]:
# imports
from acquire import clean_zillow

In [20]:
# getting my clean data function 'clean_zillow' from my acquire file
df = clean_zillow(df)

In [21]:
df.head() # check_yo_head

,sqft,baths,beds,tax_value,year_built,tax_amount,fips
parcel_id,,,,,,,
14634203,1125.0,2.0,3.0,289445.0,1974.0,3490.16,6059.0
11721753,1316.0,2.0,3.0,205123.0,1923.0,2627.48,6037.0
11289917,1458.0,2.0,3.0,136104.0,1970.0,2319.90,6037.0
11637029,1766.0,2.0,3.0,810694.0,1980.0,9407.01,6037.0
11705026,1421.0,1.0,2.0,35606.0,1911.0,543.69,6037.0


In [23]:
# taking a look at rows and columns after cleaning data, I now have 7 columns 
df.shape

(38508, 7)

In [24]:
# some quick stats about the clean zillow data
df.describe()

,sqft,baths,beds,tax_value,year_built,tax_amount,fips
count,38508.000000,38508.000000,38508.000000,3.850800e+04,38480.000000,38507.000000,38508.000000
mean,1762.761400,2.280202,3.019087,4.952885e+05,1969.514137,6027.063675,6049.890153
std,928.307857,0.944140,1.018296,6.288708e+05,23.056181,7312.638119,21.414738
min,240.000000,0.000000,0.000000,1.000000e+03,1878.000000,19.920000,6037.000000
25%,1174.000000,2.000000,2.000000,2.091238e+05,1954.000000,2723.745000,6037.000000
50%,1525.000000,2.000000,3.000000,3.636145e+05,1971.000000,4464.950000,6037.000000
75%,2082.000000,3.000000,4.000000,5.771802e+05,1987.000000,6980.680000,6059.000000
max,17245.000000,11.000000,12.000000,2.385837e+07,2016.000000,276797.830000,6111.000000
